In [24]:
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import numpy as np
from rake_nltk import Rake
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
pd.options.mode.chained_assignment = None  # default='warn'
df2=pd.read_csv
df1= pd.read_csv("dataset/animes.csv")
df1.head()

In [ ]:
df=df1[['title','synopsis','genre','episodes','ranked','popularity','score']]
df['synopsis']=df['synopsis'].fillna(" ")
df['synopsis']=df['synopsis'].apply(str)
df.head()


In [ ]:

df.dropna(inplace=True)

blanks = []  # start with an empty list

col=['title','synopsis','genre','episodes','score']
for i,col in df.iterrows():  # iterate over the DataFrame
    if type(col)==str:            # avoid NaN values
        if col.isspace():         # test 'review' for whitespace
            blanks.append(i)     # add matching index numbers to the list

df.drop(blanks, inplace=True)




In [ ]:
# initializing the new column
df['Key_words'] = " "

for index, row in df.iterrows():
    description = row['synopsis']
    # instantiating Rake, by default it uses english stopwords from NLTK
    # and discards all puntuation characters as well
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(description)
    

    # getting the dictionary whith key words as keys and their scores as values
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column for the corresponding movie
    df.at[index, 'Key_words'] = list((key_words_dict_scores.keys()))


In [ ]:
df.head()
# print(type(df['genre'][0]))

In [ ]:
df["bag_of_words"]=""

genre = ['genre']
keyword= ['Key_words']
scores=['episodes','score']

for index, row in df.iterrows():
    words = ''
    for col in genre:
        row[col] = row[col].replace('[', '').replace(']', '').replace('\'', '')
        row[col] = row[col].replace(',', ' ')
        words = words + ''.join(row[col])+' '
    for col in keyword:
        words = words + ' '.join(row[col])+ ' '
    for col in scores:
        words = words + str(int(row[col]))
    df.at[index,'bag_of_words'] = words.lower()
df.head()

In [ ]:
df["bag_of_words"].head()

df1=df[['title','bag_of_words']]

# Convert lists to tuples in the 'title' column

# Remove duplicate rows
df1 = df1.drop_duplicates()



In [ ]:
count = CountVectorizer()
count_matrix = count.fit_transform(df1['bag_of_words'])

indices = pd.Series(df1['title'])
print(indices[:5]) 
#index to access to get recommendation
cosine_sim = cosine_similarity(count_matrix, count_matrix)


In [ ]:

def recommendations(Title, cosine_sim = cosine_sim):
    
    recommended_anime = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == Title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = True)

    # getting the indexes of the 10 most similar movies
    top_5_indexes = list(score_series.iloc[1:6].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_5_indexes:
            recommended_anime.append(list(df1.title)[i])
        
    return recommended_anime

In [ ]:
df1.head()

In [ ]:
recommendations('Haikyuu!! Second Season')

In [ ]:
df[df['title']=='Kiki to Lala no Ohimesama ni Naritai']

In [ ]:
print(df[['bag_of_words'][df['title']=='Kiki to Lala no Ohimesama ni Naritai']])